<a href="https://colab.research.google.com/github/obeabi/SdsOctober2020Challenge/blob/main/CategoricalCols_Main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Written by Abiola Obembe
# SDS October Challenge #2 - Used Car Prices
## 2021-01-09

## Feature engineering of categorical columns

This month you have been hired by a new car trader company to help sell its used cars. You will be analyzing used cars from multiple manufacturers and of different models. Your job is to help the car trader company determine the price of its used cars.

## Step 1: Data Preprocessing 

In [77]:
# Download packages
!pip install category_encoders

In [78]:
# Importing the libraries
import pickle
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from scipy import stats
from scipy.stats import f_oneway
from scipy.stats import  zscore
import category_encoders as ce
from sklearn.preprocessing import StandardScaler 
from sklearn.linear_model  import Ridge,Lasso,RidgeCV, LassoCV, ElasticNet, ElasticNetCV, LinearRegression
from sklearn.model_selection import train_test_split

%matplotlib inline
plt.rcParams['figure.figsize'] = (8.0, 6.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

print('Libraries installed successfully!')

np.random.seed(0)

Libraries installed successfully!


In [79]:
# Suppressing warnings
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [80]:
# Importing the training dataset

df_train = pd.read_csv('Train.csv', engine= 'python', encoding= 'latin1')
df_train.drop('Unnamed: 0', axis = 1, inplace = True)

df_train.head()

,manufacturer_name,model_name,transmission,color,odometer_value,year_produced,engine_fuel,engine_has_gas,engine_type,engine_capacity,body_type,has_warranty,state,drivetrain,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,duration_listed,price_usd
0,Ford,Fusion,mechanical,blue,245000,2006,gasoline,0,gasoline,1.6,hatchback,0,owned,front,1,0,0,0,0,0,0,0,0,0,7,4250.0
1,Dodge,Caravan,automatic,silver,265542,2002,gasoline,0,gasoline,3.3,minivan,0,owned,front,0,1,0,0,0,0,0,0,0,0,133,4000.0
2,Ford,Galaxy,mechanical,blue,168000,2009,diesel,0,diesel,1.8,minivan,0,owned,front,0,0,0,1,0,0,1,1,1,1,0,10900.0
3,Mazda,6,mechanical,other,225522,2008,gasoline,0,gasoline,1.8,universal,0,owned,front,0,1,1,0,0,1,0,0,1,1,20,6999.0
4,Audi,80,mechanical,black,370000,1991,gasoline,0,gasoline,1.8,sedan,0,owned,front,0,0,0,0,0,0,0,0,0,1,160,1600.0


In [81]:
# priniting the details about the train dataset

print("\nEDA on Training Set\n")
print("#"*30)

print("\nFeatures/Columns : \n", df_train.columns)
print("\n\nNumber of Features/Columns : ", len(df_train.columns))
print("\nNumber of Rows : ",len(df_train))
print("\n\nData Types :\n", df_train.dtypes)

print("\n Contains NaN/Empty cells : ", df_train.isnull().values.any())

print("\n Total empty cells by column :\n", df_train.isnull().sum(), "\n\n")

print("\n Perecentage of missing values in train datframe is: \n", str(100*(df_train.isnull().sum().sum()/len(df_train)))+'%')


EDA on Training Set

##############################

Features/Columns : 
 Index(['manufacturer_name', 'model_name', 'transmission', 'color',
       'odometer_value', 'year_produced', 'engine_fuel', 'engine_has_gas',
       'engine_type', 'engine_capacity', 'body_type', 'has_warranty', 'state',
       'drivetrain', 'feature_0', 'feature_1', 'feature_2', 'feature_3',
       'feature_4', 'feature_5', 'feature_6', 'feature_7', 'feature_8',
       'feature_9', 'duration_listed', 'price_usd'],
      dtype='object')


Number of Features/Columns :  26

Number of Rows :  30824


Data Types :
 manufacturer_name     object
model_name            object
transmission          object
color                 object
odometer_value         int64
year_produced          int64
engine_fuel           object
engine_has_gas         int64
engine_type           object
engine_capacity      float64
body_type             object
has_warranty           int64
state                 object
drivetrain            object
fe

In [82]:
# Importing the test dataset

df_test = pd.read_csv('Test.csv', engine= 'python', encoding= 'latin1')
df_test.drop('Unnamed: 0', axis = 1, inplace = True)
df_test.head()

,manufacturer_name,model_name,transmission,color,odometer_value,year_produced,engine_fuel,engine_has_gas,engine_type,engine_capacity,body_type,has_warranty,state,drivetrain,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,duration_listed
0,Renault,Megane,mechanical,blue,360000,1997,gasoline,0,gasoline,1.6,coupe,0,owned,front,0,1,0,0,0,0,0,1,0,1,114
1,Peugeot,206,mechanical,silver,267000,1999,gasoline,0,gasoline,1.4,hatchback,0,owned,front,0,0,0,0,0,0,0,0,0,1,67
2,Volkswagen,Sharan,mechanical,blue,172000,2000,gasoline,0,gasoline,2.0,minivan,0,owned,front,1,0,0,0,0,0,0,0,0,0,50
3,Volvo,XC60,mechanical,white,230000,2009,diesel,0,diesel,2.4,universal,0,owned,front,0,1,1,1,0,1,1,1,1,1,79
4,Mazda,3,mechanical,silver,206000,2007,gasoline,0,gasoline,1.6,sedan,0,owned,front,0,1,0,1,0,1,0,0,0,0,74


In [83]:
# priniting the details about the test dataset

print("#"*30)
print("\nEDA on Test Set\n")
print("#"*30)


print("\nFeatures/Columns : \n",df_test.columns)
print("\n\nNumber of Features/Columns : ",len(df_test.columns))
print("\nNumber of Rows : ",len(df_test))
print("\n\nData Types :\n", df_test.dtypes)
print("\n Contains NaN/Empty cells : ", df_test.isnull().values.any())
print("\n Total empty cells by column :\n", df_test.isnull().sum())
print("\n Perecentage of missing values in test datframe is: \n", str(100*(df_test.isnull().sum().sum()/len(df_test)))+'%')

##############################

EDA on Test Set

##############################

Features/Columns : 
 Index(['manufacturer_name', 'model_name', 'transmission', 'color',
       'odometer_value', 'year_produced', 'engine_fuel', 'engine_has_gas',
       'engine_type', 'engine_capacity', 'body_type', 'has_warranty', 'state',
       'drivetrain', 'feature_0', 'feature_1', 'feature_2', 'feature_3',
       'feature_4', 'feature_5', 'feature_6', 'feature_7', 'feature_8',
       'feature_9', 'duration_listed'],
      dtype='object')


Number of Features/Columns :  25

Number of Rows :  7707


Data Types :
 manufacturer_name     object
model_name            object
transmission          object
color                 object
odometer_value         int64
year_produced          int64
engine_fuel           object
engine_has_gas         int64
engine_type           object
engine_capacity      float64
body_type             object
has_warranty           int64
state                 object
drivetrain        

In [84]:
# Extract numerical columns from train dataframe
numerical_cols = [cname for cname in df_train.columns if
                df_train[cname].dtype in ['int64', 'float64']]

num_train= df_train[numerical_cols].copy()

num_train.head()

,odometer_value,year_produced,engine_has_gas,engine_capacity,has_warranty,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,duration_listed,price_usd
0,245000,2006,0,1.6,0,1,0,0,0,0,0,0,0,0,0,7,4250.0
1,265542,2002,0,3.3,0,0,1,0,0,0,0,0,0,0,0,133,4000.0
2,168000,2009,0,1.8,0,0,0,0,1,0,0,1,1,1,1,0,10900.0
3,225522,2008,0,1.8,0,0,1,1,0,0,1,0,0,1,1,20,6999.0
4,370000,1991,0,1.8,0,0,0,0,0,0,0,0,0,0,1,160,1600.0


In [85]:
# pRINT TEST SET

num_test = df_test[numerical_cols[:-1]]
num_test.head()

,odometer_value,year_produced,engine_has_gas,engine_capacity,has_warranty,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,duration_listed
0,360000,1997,0,1.6,0,0,1,0,0,0,0,0,1,0,1,114
1,267000,1999,0,1.4,0,0,0,0,0,0,0,0,0,0,1,67
2,172000,2000,0,2.0,0,1,0,0,0,0,0,0,0,0,0,50
3,230000,2009,0,2.4,0,0,1,1,1,0,1,1,1,1,1,79
4,206000,2007,0,1.6,0,0,1,0,1,0,1,0,0,0,0,74


In [86]:
# Extract categorical features from dataframe and establish train set and test set
category_cols = [col for col in df_train.columns if df_train[col].dtype == "object"]

# Frame that contains category columns and target variable
category_frame = category_cols.copy()
category_frame.append('price_usd')

# Create test and train dataframe
cat_df_train = df_train[category_cols]
cat_df_test = df_test[category_cols]

# Print dataframe
cat_df_train.head()

,manufacturer_name,model_name,transmission,color,engine_fuel,engine_type,body_type,state,drivetrain
0,Ford,Fusion,mechanical,blue,gasoline,gasoline,hatchback,owned,front
1,Dodge,Caravan,automatic,silver,gasoline,gasoline,minivan,owned,front
2,Ford,Galaxy,mechanical,blue,diesel,diesel,minivan,owned,front
3,Mazda,6,mechanical,other,gasoline,gasoline,universal,owned,front
4,Audi,80,mechanical,black,gasoline,gasoline,sedan,owned,front


In [87]:
# Let's print the test set
cat_df_test.head()

,manufacturer_name,model_name,transmission,color,engine_fuel,engine_type,body_type,state,drivetrain
0,Renault,Megane,mechanical,blue,gasoline,gasoline,coupe,owned,front
1,Peugeot,206,mechanical,silver,gasoline,gasoline,hatchback,owned,front
2,Volkswagen,Sharan,mechanical,blue,gasoline,gasoline,minivan,owned,front
3,Volvo,XC60,mechanical,white,diesel,diesel,universal,owned,front
4,Mazda,3,mechanical,silver,gasoline,gasoline,sedan,owned,front


In [88]:
# let's have a look at how many labels each variable has

for col in category_cols:
    print(col, ': ', len(cat_df_train[col].unique()), ' labels')

manufacturer_name :  55  labels
model_name :  1070  labels
transmission :  2  labels
color :  12  labels
engine_fuel :  6  labels
engine_type :  3  labels
body_type :  12  labels
state :  3  labels
drivetrain :  3  labels


In [89]:
# Let's print the shape of this train and test dataframe
print("The shape of the train set is: ", cat_df_train.shape)

print("The shape of the train set is: ", cat_df_test.shape)

The shape of the train set is:  (30824, 9)
The shape of the train set is:  (7707, 9)


In [90]:
# Columns that can be safely label encoded in validation set
good_label_cols = [col for col in category_cols if
                   set(cat_df_train[col]) == set(cat_df_test[col])]


# Problematic columns that will be dropped from the validation dataset
bad_label_cols = list(set(category_cols) - set(good_label_cols)) 

print('Categorical columns that will be label encoded :', good_label_cols)
print('\nCategorical columns that will be dropped from the dataset for label encoding:', bad_label_cols)

Categorical columns that will be label encoded : ['manufacturer_name', 'transmission', 'color', 'engine_type', 'body_type', 'state', 'drivetrain']

Categorical columns that will be dropped from the dataset for label encoding: ['model_name', 'engine_fuel']


In [91]:
# Select categorical columns with relatively low cardinality (convenient but arbitrary)
low_cardinality = [cname for cname in category_cols if
                    cat_df_train[cname].nunique() < 5 and
                    cat_df_train[cname].dtype == "object"]
low_cardinality

['transmission', 'engine_type', 'state', 'drivetrain']

In [92]:
# Select categorical columns with relatively low cardinality (convenient but arbitrary)
high_cardinality = [cname for cname in category_cols if
                    cat_df_train[cname].nunique() > 5 and
                    cat_df_train[cname].dtype == "object"]
high_cardinality

['manufacturer_name', 'model_name', 'color', 'engine_fuel', 'body_type']

### Method 1: One-hot encoding of all columns

In [93]:
# let's examine how many columns we will obtain after one hot encoding these variables
cat_df_train_oh = pd.get_dummies(cat_df_train, drop_first=True)
cat_df_test_oh = pd.get_dummies(cat_df_test, drop_first=True)

# Let's print the shape of this train and test dataframe after one-hot encoding
print("The shape of the train set is: ", cat_df_train_oh.shape)

print("The shape of the test set is: ", cat_df_test_oh.shape)

cat_df_train_oh.to_csv('df_train_oh.csv')
cat_df_test_oh.to_csv('df_test_oh.csv')


The shape of the train set is:  (30824, 1157)
The shape of the test set is:  (7707, 907)


In [94]:
cat_df_train_oh.head()

,manufacturer_name_Alfa Romeo,manufacturer_name_Audi,manufacturer_name_BMW,manufacturer_name_Buick,manufacturer_name_Cadillac,manufacturer_name_Chery,manufacturer_name_Chevrolet,manufacturer_name_Chrysler,manufacturer_name_Citroen,manufacturer_name_Dacia,manufacturer_name_Daewoo,manufacturer_name_Dodge,manufacturer_name_Fiat,manufacturer_name_Ford,manufacturer_name_Geely,manufacturer_name_Great Wall,manufacturer_name_Honda,manufacturer_name_Hyundai,manufacturer_name_Infiniti,manufacturer_name_Iveco,manufacturer_name_Jaguar,manufacturer_name_Jeep,manufacturer_name_Kia,manufacturer_name_LADA,manufacturer_name_Lancia,manufacturer_name_Land Rover,manufacturer_name_Lexus,manufacturer_name_Lifan,manufacturer_name_Lincoln,manufacturer_name_Mazda,manufacturer_name_Mercedes-Benz,manufacturer_name_Mini,manufacturer_name_Mitsubishi,manufacturer_name_Nissan,manufacturer_name_Opel,manufacturer_name_Peugeot,manufacturer_name_Pontiac,manufacturer_name_Porsche,manufacturer_name_Renault,manufacturer_name_Rover,...,model_name_ÃÂÃÂÃÂ¡,model_name_ÃÂÃÂÃÂ¸ÃÂ´ÃÂ¾ÃÂ,model_name_ÃÂ20,model_name_ÃÂ5,model_name_ÃÂ¡ÃÂ¾ÃÂ±ÃÂ¾ÃÂ»ÃÂ,model_name_ÃÂ¢ÃÂ°ÃÂ²ÃÂÃÂ¸ÃÂ,transmission_mechanical,color_blue,color_brown,color_green,color_grey,color_orange,color_other,color_red,color_silver,color_violet,color_white,color_yellow,engine_fuel_electric,engine_fuel_gas,engine_fuel_gasoline,engine_fuel_hybrid-diesel,engine_fuel_hybrid-petrol,engine_type_electric,engine_type_gasoline,body_type_coupe,body_type_hatchback,body_type_liftback,body_type_limousine,body_type_minibus,body_type_minivan,body_type_pickup,body_type_sedan,body_type_suv,body_type_universal,body_type_van,state_new,state_owned,drivetrain_front,drivetrain_rear
0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,1,1,0
1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,1,1,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,1,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,1,1,0
4,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,1,1,0


#### The number of columns generated is too much and would result in the curse of dimensionality and it may be lead to overfitting since there are way too much columns

### Method 2: Count Encoding

In [95]:
# Create ne dataframe for train and test set
cat_df_train_ce = cat_df_train.copy()
cat_df_test_ce = cat_df_test.copy()

cat_df_train_ce.head()

,manufacturer_name,model_name,transmission,color,engine_fuel,engine_type,body_type,state,drivetrain
0,Ford,Fusion,mechanical,blue,gasoline,gasoline,hatchback,owned,front
1,Dodge,Caravan,automatic,silver,gasoline,gasoline,minivan,owned,front
2,Ford,Galaxy,mechanical,blue,diesel,diesel,minivan,owned,front
3,Mazda,6,mechanical,other,gasoline,gasoline,universal,owned,front
4,Audi,80,mechanical,black,gasoline,gasoline,sedan,owned,front


In [96]:
# Performing count encoding

count_enc = ce.CountEncoder()

# Apply the count encoder to dataframe
count_encoded_train = count_enc.fit_transform(cat_df_train_ce).add_suffix('_ce')
count_encoded_test = count_enc.fit_transform(cat_df_test_ce).add_suffix('_ce')

count_encoded_train.head()

,manufacturer_name_ce,model_name_ce,transmission_ce,color_ce,engine_fuel_ce,engine_type_ce,body_type_ce,state_ce,drivetrain_ce
0,2080,119,20493,4671,19259,20511,6095,30154,22193
1,247,69,10331,5480,19259,20511,2916,30154,22193
2,2080,153,20493,4671,10303,10305,2916,30154,22193
3,1081,223,20493,2141,19259,20511,4397,30154,22193
4,1972,234,20493,6162,19259,20511,10433,30154,22193


In [97]:
# Let's print the test set
count_encoded_test.head()

,manufacturer_name_ce,model_name_ce,transmission_ce,color_ce,engine_fuel_ce,engine_type_ce,body_type_ce,state_ce,drivetrain_ce
0,494,75,5140,1131,4806,5136,148,7569,5553
1,380,29,5140,1372,4806,5136,1549,7569,5553
2,837,33,5140,1131,4806,5136,692,7569,5553
3,148,11,5140,822,2569,2569,1110,7569,5553
4,247,37,5140,1372,4806,5136,2578,7569,5553


In [98]:
# We collect all of these features (numerical and categorical) in a new dataframe that we can use to train a count encoder-based model
df_train_ce = num_train.join(count_encoded_train)
df_test_ce = num_test.join(count_encoded_test)

df_train_ce.to_csv('df_train_ce.csv')
df_test_ce.to_csv('df_test_ce.csv')

df_train_ce.head()

,odometer_value,year_produced,engine_has_gas,engine_capacity,has_warranty,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,duration_listed,price_usd,manufacturer_name_ce,model_name_ce,transmission_ce,color_ce,engine_fuel_ce,engine_type_ce,body_type_ce,state_ce,drivetrain_ce
0,245000,2006,0,1.6,0,1,0,0,0,0,0,0,0,0,0,7,4250.0,2080,119,20493,4671,19259,20511,6095,30154,22193
1,265542,2002,0,3.3,0,0,1,0,0,0,0,0,0,0,0,133,4000.0,247,69,10331,5480,19259,20511,2916,30154,22193
2,168000,2009,0,1.8,0,0,0,0,1,0,0,1,1,1,1,0,10900.0,2080,153,20493,4671,10303,10305,2916,30154,22193
3,225522,2008,0,1.8,0,0,1,1,0,0,1,0,0,1,1,20,6999.0,1081,223,20493,2141,19259,20511,4397,30154,22193
4,370000,1991,0,1.8,0,0,0,0,0,0,0,0,0,0,1,160,1600.0,1972,234,20493,6162,19259,20511,10433,30154,22193


### Method 3 : Target Encoding

In [99]:
# Create the  Target encoder object
cat_df_train_te = cat_df_train.copy()
cat_df_test_te = cat_df_test.copy()

# Apply traget encoding 
target_enc = ce.TargetEncoder(cols=category_cols)
target_enc.fit(cat_df_train_te[category_cols],num_train['price_usd'] )

# Transform the features, rename the columns with _target suffix, and join to dataframe
target_encoded_train = target_enc.transform(cat_df_train_te[category_cols]).add_suffix('_te')
target_encoded_test = target_enc.transform(cat_df_test_te[category_cols]).add_suffix('_te')

# We collect all of these features (numerical and categorical) in a new dataframe that we can use to train a Target encoder-based model
df_train_te = num_train.join(target_encoded_train)
df_test_te = num_test.join(target_encoded_test)

df_train_te.to_csv('df_train_te.csv')
df_test_te.to_csv('df_test_te.csv')

df_train_te.head()

,odometer_value,year_produced,engine_has_gas,engine_capacity,has_warranty,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,duration_listed,price_usd,manufacturer_name_te,model_name_te,transmission_te,color_te,engine_fuel_te,engine_type_te,body_type_te,state_te,drivetrain_te
0,245000,2006,0,1.6,0,1,0,0,0,0,0,0,0,0,0,7,4250.0,5029.863851,9575.330756,4470.104020,4786.850334,6270.954384,6222.792804,4038.937757,6460.257538,5373.827909
1,265542,2002,0,3.3,0,0,1,0,0,0,0,0,0,0,0,133,4000.0,5735.589271,3179.271449,10916.245249,5934.723934,6270.954384,6222.792804,6182.120881,6460.257538,5373.827909
2,168000,2009,0,1.8,0,0,0,0,1,0,0,1,1,1,1,0,10900.0,5029.863851,5543.469085,4470.104020,4786.850334,7433.666811,7432.738394,6182.120881,6460.257538,5373.827909
3,225522,2008,0,1.8,0,0,1,1,0,0,1,0,0,1,1,20,6999.0,4735.107530,8128.120448,4470.104020,5854.666404,6270.954384,6222.792804,4971.458897,6460.257538,5373.827909
4,370000,1991,0,1.8,0,0,0,0,0,0,0,0,0,0,1,160,1600.0,7121.501476,1643.286410,4470.104020,9160.871863,6270.954384,6222.792804,5788.889002,6460.257538,5373.827909


In [100]:
print(df_train_te.isnull().sum().sum())
print(df_test_te.isnull().sum().sum())

0
0
